In [ ]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR
import numpy as np
import pickle

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784, 32)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(32, 10)

    def forward(self, x):
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

In [ ]:
model = Net()

In [ ]:
print("> printing the model")
print(model)

> printing the model
Net(
  (fc1): Linear(in_features=784, out_features=32, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc2): Linear(in_features=32, out_features=10, bias=True)
)


In [ ]:
def train(args, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        #print("Prediction", model()
        optimizer.step()
        if batch_idx % args['log_interval'] == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
            if args['dry_run']:
                break


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    

def run():
    args = {
        'no_cuda' : True,
        'seed' : 1,
        'batch_size' : 64,
        'test_batch_size' : 1000,
        'lr' : 1.0,
        'gamma' : 0.7,
        'epochs' : 5,
        'save_model' : True,
        'log_interval' : 10,
        'dry_run' : False
        
    }
   
    use_cuda = not args['no_cuda'] and torch.cuda.is_available()

    torch.manual_seed(args['seed'])

    device = torch.device("cuda" if use_cuda else "cpu")

    train_kwargs = {'batch_size': args['batch_size']}
    test_kwargs = {'batch_size': args['test_batch_size']}
    if use_cuda:
        cuda_kwargs = {'num_workers': 1,
                       'pin_memory': True,
                       'shuffle': True}
        train_kwargs.update(cuda_kwargs)
        test_kwargs.update(cuda_kwargs)

    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
        ])
    dataset1 = datasets.MNIST('../data', train=True, download=True,
                       transform=transform)
    dataset2 = datasets.MNIST('../data', train=False,
                       transform=transform)
    train_loader = torch.utils.data.DataLoader(dataset1,**train_kwargs)
    test_loader = torch.utils.data.DataLoader(dataset2, **test_kwargs)

    model = Net().to(device)
    optimizer = optim.Adadelta(model.parameters(), lr=args['lr'])

    scheduler = StepLR(optimizer, step_size=1, gamma=args['gamma'])
    for epoch in range(1, args['epochs'] + 1):
        train(args, model, device, train_loader, optimizer, epoch)
        test(model, device, test_loader)
        scheduler.step()

    if args['save_model']:
        torch.save(model.state_dict(), "mnist_dnn.pt")

In [ ]:
run()

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.329856
Train Epoch: 1 [640/60000 (1%)]	Loss: 1.573720
Train Epoch: 1 [1280/60000 (2%)]	Loss: 1.413218
Train Epoch: 1 [1920/60000 (3%)]	Loss: 0.955432
Train Epoch: 1 [2560/60000 (4%)]	Loss: 0.786503
Train Epoch: 1 [3200/60000 (5%)]	Loss: 0.881180
Train Epoch: 1 [3840/60000 (6%)]	Loss: 0.771456
Train Epoch: 1 [4480/60000 (7%)]	Loss: 0.804304
Train Epoch: 1 [5120/60000 (9%)]	Loss: 1.143172
Train Epoch: 1 [5760/60000 (10%)]	Loss: 0.575277
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.671376
Train Epoch: 1 [7040/60000 (12%)]	Loss: 0.671880
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.625420
Train Epoch: 1 [8320/60000 (14%)]	Loss: 0.604873
Train Epoch: 1 [8960/60000 (15%)]	Loss: 0.773807
Train Epoch: 1 [9600/60000 (16%)]	Loss: 0.766047
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.657893
Train Epoch: 1 [10880/60000 (18%)]	Loss: 0.557043
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.810708
Train Epoch: 1 [12160/60000 (20%)]	Loss: 0.812729
Train Epoch: 1 [12800/60000 (

In [ ]:
#exploration
model = Net()
model.load_state_dict(torch.load("mnist_dnn.pt"))
model.eval()
b1 = model.fc1.bias.tolist()
w1 = model.fc1.weight.tolist()
b2 = model.fc2.bias.tolist()
w2 = model.fc2.weight.tolist()

In [ ]:
!ls


mnist_dnn.pt  sample_data


In [ ]:
#LOAD AN EXAMPLE NUMBER TO TEST MANUAL INFERENCE

In [ ]:
transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
        ])
ds = datasets.MNIST('../data', train=True, download=True,
                       transform=transform)
IDX = 1

#print("Prediction", model(ds))

d = ds.data[IDX].tolist()
d_flatten = []
for i in range(28):
    for j in range(28):
        print("%3d" % d[i][j], end=" ")
        d_flatten.append(d[i][j])
    print()

  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0 
  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0 
  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0 
  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0 
  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0  51 159 253 159  50   0   0   0   0   0   0   0   0 
  0   0   0   0   0   0   0   0   0   0   0   0   0   0  48 238 252 252 252 237   0   0   0   0   0   0   0   0 
  0   0   0   0   0   0   0   0   0   0   0   0   0  54 227 253 252 239 233 252  57   6   0   0   0   0   0   0 
  0   0   0   0   0   0   0   0   0   0   0  10  60 224 252 253 252 202  84 252 253 122   0   0   0   0   0   0 
  0   0   0   0   0   0   0   0   0   0   0 163 252 252 252 253 252 252  96 189 253 167   0   0 

In [ ]:
#Function FC1
def FC1(x):
  sum = [0 for i in range(32)]
  for k in range(32):
    for i in range(784):
      sum[k] += x[i]*w1[k][i]
    
    sum[k] = sum[k] + b1[k]

  return sum    

#RELU Function
def RELU(fc1_out):
  for k in range(32):
    if fc1_out[k]<0 :
      fc1_out[k] = 0

#Function FC2
def FC2(x):
  sum = [0 for j in range(10)]
  
  for k in range(10):
    for i in range(32):
      sum[k] += x[i]*w2[k][i]
    
    sum[k] = sum[k] + b2[k]

  return sum

In [ ]:

#test FC1
FC1_output = FC1(d_flatten)

In [ ]:
RELU(FC1_output)

In [ ]:
FC2_output = FC2(FC1_output)
FC2_output

[408.01022160954466,
 -2881.1622131022145,
 -459.87114460462044,
 -805.9846144106874,
 -2202.5064838162857,
 -560.3629856452632,
 -1136.551053829702,
 -1697.7894476289957,
 -1024.4101038761607,
 -1066.8907916270775]

In [ ]:
solution = np.argmax(FC2_output)
solution

0

Net(
  (fc1): Linear(in_features=784, out_features=32, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc2): Linear(in_features=32, out_features=10, bias=True)
)

In [ ]:
test(model, torch.device("cpu"), test_loader)


Test set: Average loss: 0.4822, Accuracy: 8639/10000 (86%)

